In [ ]:
import os

api_key = os.environ.get("GEMINI_API_KEY")

In [2]:
!pip install google-generativeai
!pip install google-api-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 3.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.4
    Uninstalling protobuf-4.24.4:
      Successfully uninstalled protobuf-4.24.4
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.0
    Uninstalling grpcio-1.59.0:
      Successfully uninstalled grpcio-1.59.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.59.0 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 5.29.4 which is incompatible.
tensorflow-macos 2.13.0rc1 requires keras<2.14,>=2.13.1rc0, but you have keras 3.7.0 which is incompatible.
tensorflow-macos 2.13.0rc1 requi

In [ ]:
import os
import json
import time
from multiprocessing.pool import Pool
import threading

import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted

output_dir = "persian_recommendation_output"
os.makedirs(output_dir, exist_ok=True)

# Safety settings to ensure content generation isn't blocked
SAFETY_CONFIG = [
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
]


def load_model(api_key):
    """Initialize and return the Gemini model with the provided API key."""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel("gemini-2.0-flash")
    return model


def get_recommendations(request, model, api_key):
    """
    Generate recommendations based on the request.

    Args:
        request: A dictionary containing 'entity', 'genre', and optionally 'example'
        model: The Gemini model to use
        api_key: The API key for fallback in case of rate limiting

    Returns:
        A tuple of (recommendations_list, raw_response)
    """
    entity = request.get("entity", "movies")
    genre = request.get("genre", "")

    prompt = f"""شما یک موتور پیشنهاددهنده هستید. آیا می‌توانید فهرستی از ۱۰ {genre} {entity} خوب به من پیشنهاد دهید؟

        فقط با یک آرایه JSON از پیشنهادها در این قالب دقیق پاسخ دهید:
        [
        {{
        "name": "<نام مورد پیشنهادی اول>",
        "language": "<زبان مورد اول>",
        "country": "<کشور سازنده مورد اول>",
        "genre": "<ژانر مورد اول>"
        }},
        ... و به همین صورت برای هر ۱۰ مورد
        ]
        """

    def do_request():
        try:
            chat_completion = model.generate_content(
                prompt, safety_settings=SAFETY_CONFIG
            )

            if (
                str(chat_completion.prompt_feedback).strip()
                == "block_reason: OTHER".strip()
            ):
                return None, "Content was blocked due to safety settings."

            # Extract the JSON array from the response
            response_text = chat_completion.text

            # Find JSON content within the response (it might be wrapped in markdown code blocks)
            if "```json" in response_text:
                json_content = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text:
                json_content = response_text.split("```")[1].split("```")[0].strip()
            else:
                json_content = response_text.strip()

            # Parse the JSON response
            try:
                recommendations = json.loads(json_content)
                # Ensure it's a list with the correct structure
                if not isinstance(recommendations, list):
                    return None, f"Invalid response format: {response_text}"

                for item in recommendations:
                    if not all(key in item for key in ["name", "language", "country"]):
                        return (
                            None,
                            f"Missing required fields in response: {response_text}",
                        )

                # Wait to avoid hitting rate limits
                time.sleep(0.5)
                return recommendations, response_text
            except json.JSONDecodeError:
                return None, f"Failed to parse JSON response: {response_text}"

        except Exception as e:
            return None, f"Error generating recommendations: {str(e)}"

    try:
        return do_request()
    except ResourceExhausted:
        print(f"Rate limit exceeded for API key: {api_key}")
        time.sleep(60)  # Wait before retrying
        return do_request()
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return None, str(e)


def main():
    """Main function to process recommendation requests."""

    genres = [
        "Docu",
        "Action",
        "Drama",
        "Horror",
        "Fantasy",
        "Romance",
        "Mystery",
        "Thriller",
        "Comedy",
        "Sci-Fi",
        "",
    ]

    example_requests = [{"entity": "movies", "genre": genre} for genre in genres]

    model = load_model(api_key)

    for i, request in enumerate(example_requests):
        request_id = f"request_{i}"
        example_requests[i]["request_id"] = request_id
        recommendations, raw_response = get_recommendations(request, model, api_key)
        result = {
            "request_id": request_id,
            "request": request,
            "recommendations": recommendations,
            "raw_response": raw_response,
        }
        with open(
            os.path.join(output_dir, f"recommendations_{request_id}.json"), "a"
        ) as f:
            f.write(json.dumps(result) + "\n")


if __name__ == "__main__":
    main()